In [1]:
from wequant import *
from wequant.wefetch import *
from wequant.wefetch.query import *
from wequant.wefetch.query_ad import *
from wequant.mongo import get_db
import pandas as pd


In [ ]:
db = get_db()
CANDIDATE_CODE_FIELDS = ['code', 'symbol', 'ts_code', 'a_stockcode']

def _ensure_collection(name: str):
    if name not in db.list_collection_names():
        print(f'skip: missing collection {name}')
        return None
    coll = db[name]
    if coll.find_one({}, {'_id': 1}) is None:
        print(f'skip: empty collection {name}')
        return None
    return coll

def _sample_code(coll_name: str):
    coll = _ensure_collection(coll_name)
    if coll is None:
        return None, None
    doc = coll.find_one({}, {field: 1 for field in CANDIDATE_CODE_FIELDS})
    if not doc:
        return None, None
    for field in CANDIDATE_CODE_FIELDS:
        val = doc.get(field)
        if val not in (None, ''):
            return str(val), field
    return None, None

def _date_range(coll_name: str, code=None, field=None, prefer_time: bool = False):
    coll = _ensure_collection(coll_name)
    if coll is None:
        return None, None

    if coll.find_one({'date_stamp': {'$exists': True}}, {'date_stamp': 1}):
        sort_field = 'date_stamp'
        to_dt = lambda doc: pd.to_datetime(doc[sort_field], unit='s')
    elif coll.find_one({'time_stamp': {'$exists': True}}, {'time_stamp': 1}):
        sort_field = 'time_stamp'
        to_dt = lambda doc: pd.to_datetime(doc[sort_field], unit='s')
    elif coll.find_one({'datetime': {'$exists': True}}, {'datetime': 1}):
        sort_field = 'datetime'
        to_dt = lambda doc: pd.to_datetime(doc[sort_field])
    elif coll.find_one({'date': {'$exists': True}}, {'date': 1}):
        sort_field = 'date'
        to_dt = lambda doc: pd.to_datetime(doc[sort_field])
    elif coll.find_one({'updateDate': {'$exists': True}}, {'updateDate': 1}):
        sort_field = 'updateDate'
        to_dt = lambda doc: pd.to_datetime(doc[sort_field])
    elif coll.find_one({'dir_dcl_date': {'$exists': True}}, {'dir_dcl_date': 1}):
        sort_field = 'dir_dcl_date'
        to_dt = lambda doc: pd.to_datetime(doc[sort_field])
    else:
        print(f'skip: cannot infer time field for {coll_name}')
        return None, None

    query = {}
    if code is not None:
        query[field or 'code'] = code

    first = coll.find_one(query, sort=[(sort_field, 1)])
    last = coll.find_one(query, sort=[(sort_field, -1)])
    if not first or not last:
        print(f'skip: no records in {coll_name} for sampled code')
        return None, None

    start = to_dt(first)
    end = to_dt(last)
    if prefer_time:
        return start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S')
    return start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')

def _pick_latest_collection(prefix: str):
    names = [n for n in db.list_collection_names() if n.startswith(prefix)]
    if not names:
        return None
    return sorted(names)[-1]



In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_day
# new: wequant.wefetch.fetch_stock_day
code, field = _sample_code('stock_day')
start, end = _date_range('stock_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in stock_day')
else:
    fetch_stock_day(code, start, end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_hkstock_day
# new: wequant.wefetch.fetch_hkstock_day
code, field = _sample_code('hkstock_day')
start, end = _date_range('hkstock_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in hkstock_day')
else:
    # code checking: $in expects an array
    code = [code] if isinstance(code, str) else code
    fetch_hkstock_day(code, start, end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_dk_data
# new: wequant.wefetch.fetch_dk_data
code, field = _sample_code('dk_data')
start, end = _date_range('dk_data', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in dk_data')
else:
    # code checking: $in expects an array
    code = [code] if isinstance(code, str) else code
    fetch_dk_data(code, start, end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_adj
# new: wequant.wefetch.fetch_stock_adj
code, field = _sample_code('stock_adj')
start, end = _date_range('stock_adj', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in stock_adj')
else:
    fetch_stock_adj(code, start, end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_realtime_min
# new: wequant.wefetch.fetch_stock_realtime_min
coll_name = _pick_latest_collection('realtime_kline_')
if not coll_name:
    print('skip: no realtime_kline_* collections')
else:
    code, field = _sample_code(coll_name)
    if not code:
        print(f'skip: no code in {coll_name}')
    else:
        fetch_stock_realtime_min(code, format='pd', frequence='1min', collections=db[coll_name])


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_min
# new: wequant.wefetch.fetch_stock_min
code, field = _sample_code('stock_min')
start, end = _date_range('stock_min', code, field, prefer_time=True)
freq = (db['stock_min'].find_one({'code': code}, {'type': 1}) or {}).get('type', '1min') if code else '1min'
if not code or not start or not end:
    print('skip: insufficient data in stock_min')
else:
    fetch_stock_min(code, start, end, frequence=freq, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_transaction
# new: wequant.wefetch.fetch_stock_transaction
code, field = _sample_code('stock_transaction')
start, end = _date_range('stock_transaction', code, field, prefer_time=True)
if not code or not start or not end:
    print('skip: insufficient data in stock_transaction')
else:
    fetch_stock_transaction(code, start, end, frequence='tick', format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_index_transaction
# new: wequant.wefetch.fetch_index_transaction
code, field = _sample_code('index_transaction')
start, end = _date_range('index_transaction', code, field, prefer_time=True)
if not code or not start or not end:
    print('skip: insufficient data in index_transaction')
else:
    fetch_index_transaction(code, start, end, frequence='tick', format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_trade_date
# new: wequant.wefetch.fetch_trade_date
fetch_trade_date()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_list
# new: wequant.wefetch.fetch_stock_list
fetch_stock_list()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_etf_list
# new: wequant.wefetch.fetch_etf_list
fetch_etf_list()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_index_list
# new: wequant.wefetch.fetch_index_list
fetch_index_list()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_terminated
# new: wequant.wefetch.fetch_stock_terminated
if _ensure_collection('stock_terminated') is None:
    print('skip: missing stock_terminated')
else:
    fetch_stock_terminated()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_basic_info_tushare
# new: wequant.wefetch.fetch_stock_basic_info_tushare
if _ensure_collection('stock_info_tushare') is None:
    print('skip: missing stock_info_tushare')
else:
    fetch_stock_basic_info_tushare()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_to_market_date
# new: wequant.wefetch.fetch_stock_to_market_date
if _ensure_collection('stock_info_tushare') is None:
    print('skip: missing stock_info_tushare')
else:
    code, _ = _sample_code('stock_info_tushare')
    if not code:
        print('skip: no code')
    else:
        fetch_stock_to_market_date(code)


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_full
# new: wequant.wefetch.fetch_stock_full
code, field = _sample_code('stock_day')
date, _ = _date_range('stock_day', code, field, prefer_time=False)
if not date:
    print('skip: no date in stock_day')
else:
    fetch_stock_full(date, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_index_day
# new: wequant.wefetch.fetch_index_day
code, field = _sample_code('index_day')
start, end = _date_range('index_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in index_day')
else:
    fetch_index_day(code, start, end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_index_min
# new: wequant.wefetch.fetch_index_min
code, field = _sample_code('index_min')
start, end = _date_range('index_min', code, field, prefer_time=True)
freq = (db['index_min'].find_one({'code': code}, {'type': 1}) or {}).get('type', '1min') if code else '1min'
if not code or not start or not end:
    print('skip: insufficient data in index_min')
else:
    fetch_index_min(code, start, end, frequence=freq, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_future_day
# new: wequant.wefetch.fetch_future_day
code, field = _sample_code('future_day')
start, end = _date_range('future_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in future_day')
else:
    fetch_future_day(code, start, end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_future_min
# new: wequant.wefetch.fetch_future_min
code, field = _sample_code('future_min')
start, end = _date_range('future_min', code, field, prefer_time=True)
freq = (db['future_min'].find_one({'code': code}, {'type': 1}) or {}).get('type', '1min') if code else '1min'
if not code or not start or not end:
    print('skip: insufficient data in future_min')
else:
    fetch_future_min(code, start, end, frequence=freq, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_future_list
# new: wequant.wefetch.fetch_future_list
fetch_future_list()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_ctp_future_list
# new: wequant.wefetch.fetch_ctp_future_list
print('skip: ctp collections not present in this workspace')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_future_tick
# new: wequant.wefetch.fetch_future_tick
print('skip: fetch_future_tick is NotImplemented')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_ctp_tick
# new: wequant.wefetch.fetch_ctp_tick
print('skip: ctp collections not present in this workspace')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_xdxr
# new: wequant.wefetch.fetch_stock_xdxr
code, field = _sample_code('stock_xdxr')
if not code:
    print('skip: insufficient data in stock_xdxr')
else:
    fetch_stock_xdxr(code, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_backtest_info
# new: wequant.wefetch.fetch_backtest_info
if _ensure_collection('backtest_info') is None:
    print('skip: missing backtest_info')
else:
    fetch_backtest_info()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_backtest_history
# new: wequant.wefetch.fetch_backtest_history
if _ensure_collection('backtest_history') is None:
    print('skip: missing backtest_history')
else:
    fetch_backtest_history()


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_block
# new: wequant.wefetch.fetch_stock_block
if _ensure_collection('stock_block') is None:
    print('skip: missing stock_block')
else:
    fetch_stock_block(format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_block_history
# new: wequant.wefetch.fetch_stock_block_history
code, field = _sample_code('stock_block_history')
start, end = _date_range('stock_block_history', code, field, prefer_time=False)
if not start or not end:
    print('skip: insufficient date range in stock_block_history')
elif not code:
    print('skip: no code in stock_block_history')
else:
    fetch_stock_block_history(code, start=start, end=end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_block_slice_history
# new: wequant.wefetch.fetch_stock_block_slice_history
code, field = _sample_code('stock_block_slice_history')
start, end = _date_range('stock_block_slice_history', code, field, prefer_time=False)
if not code:
    print('skip: no code in stock_block_slice_history')
else:
    fetch_stock_block_slice_history(code, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_info
# new: wequant.wefetch.fetch_stock_info
code, field = _sample_code('stock_info')
if not code:
    print('skip: insufficient data in stock_info')
else:
    fetch_stock_info(code, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_name
# new: wequant.wefetch.fetch_stock_name
code, _ = _sample_code('stock_list')
if not code:
    print('skip: insufficient data in stock_list')
else:
    fetch_stock_name(code)


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_index_name
# new: wequant.wefetch.fetch_index_name
code, _ = _sample_code('index_list')
if not code:
    print('skip: insufficient data in index_list')
else:
    fetch_index_name(code)


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_etf_name
# new: wequant.wefetch.fetch_etf_name
code, _ = _sample_code('etf_list')
if not code:
    print('skip: insufficient data in etf_list')
else:
    fetch_etf_name(code)


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_quotation
# new: wequant.wefetch.fetch_quotation
print('skip: realtime_* collections not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_quotations
# new: wequant.wefetch.fetch_quotations
print('skip: realtime_* collections not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_account
# new: wequant.wefetch.fetch_account
print('skip: account/risk/user/strategy collections not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_risk
# new: wequant.wefetch.fetch_risk
print('skip: account/risk/user/strategy collections not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_user
# new: wequant.wefetch.fetch_user
print('skip: account/risk/user/strategy collections not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_strategy
# new: wequant.wefetch.fetch_strategy
print('skip: account/risk/user/strategy collections not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_lhb
# new: wequant.wefetch.fetch_lhb
print('skip: lhb collection not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_financial_report
# new: wequant.wefetch.fetch_financial_report
print('skip: financial collection not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_financial_calendar
# new: wequant.wefetch.fetch_stock_financial_calendar
print('skip: report_calendar collection not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_stock_divyield
# new: wequant.wefetch.fetch_stock_divyield
print('skip: stock_divyield collection not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_cryptocurrency_list
# new: wequant.wefetch.fetch_cryptocurrency_list
print('skip: cryptocurrency_list collection not present')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_cryptocurrency_day
# new: wequant.wefetch.fetch_cryptocurrency_day
code, field = _sample_code('cryptocurrency_day')
start, end = _date_range('cryptocurrency_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in cryptocurrency_day')
else:
    fetch_cryptocurrency_day(code, start, end, format='pd')


In [ ]:
# old: QAFetch/QAQuery.py.QA_fetch_cryptocurrency_min
# new: wequant.wefetch.fetch_cryptocurrency_min
code, field = _sample_code('cryptocurrency_min')
start, end = _date_range('cryptocurrency_min', code, field, prefer_time=True)
freq = (db['cryptocurrency_min'].find_one({'code': code}, {'type': 1}) or {}).get('type', '1min') if code else '1min'
if not code or not start or not end:
    print('skip: insufficient data in cryptocurrency_min')
else:
    fetch_cryptocurrency_min(code, start, end, frequence=freq, format='pd')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_option_day_adv
# new: wequant.wefetch.fetch_option_day_adv
fetch_option_day_adv('000001')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_day_adv
# new: wequant.wefetch.fetch_stock_day_adv
code, field = _sample_code('stock_day')
start, end = _date_range('stock_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in stock_day')
else:
    fetch_stock_day_adv(code, start, end).to_qfq()


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_min_adv
# new: wequant.wefetch.fetch_stock_min_adv
code, field = _sample_code('stock_min')
start, end = _date_range('stock_min', code, field, prefer_time=True)
freq = (db['stock_min'].find_one({'code': code}, {'type': 1}) or {}).get('type', '1min') if code else '1min'
if not code or not start or not end:
    print('skip: insufficient data in stock_min')
else:
    fetch_stock_min_adv(code, start, end, frequence=freq).to_qfq()


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_day_full_adv
# new: wequant.wefetch.fetch_stock_day_full_adv
code, field = _sample_code('stock_day')
date, _ = _date_range('stock_day', code, field, prefer_time=False)
if not date:
    print('skip: no date in stock_day')
else:
    fetch_stock_day_full_adv(date)


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_index_day_adv
# new: wequant.wefetch.fetch_index_day_adv
code, field = _sample_code('index_day')
start, end = _date_range('index_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in index_day')
else:
    fetch_index_day_adv(code, start, end)


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_index_min_adv
# new: wequant.wefetch.fetch_index_min_adv
code, field = _sample_code('index_min')
start, end = _date_range('index_min', code, field, prefer_time=True)
freq = (db['index_min'].find_one({'code': code}, {'type': 1}) or {}).get('type', '1min') if code else '1min'
if not code or not start or not end:
    print('skip: insufficient data in index_min')
else:
    fetch_index_min_adv(code, start, end, frequence=freq)


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_transaction_adv
# new: wequant.wefetch.fetch_stock_transaction_adv
print('skip: transaction collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_index_transaction_adv
# new: wequant.wefetch.fetch_index_transaction_adv
print('skip: transaction collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_list_adv
# new: wequant.wefetch.fetch_stock_list_adv
fetch_stock_list_adv()


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_index_list_adv
# new: wequant.wefetch.fetch_index_list_adv
fetch_index_list_adv()


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_future_day_adv
# new: wequant.wefetch.fetch_future_day_adv
code, field = _sample_code('future_day')
start, end = _date_range('future_day', code, field, prefer_time=False)
if not code or not start or not end:
    print('skip: insufficient data in future_day')
else:
    fetch_future_day_adv(code, start, end)


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_future_min_adv
# new: wequant.wefetch.fetch_future_min_adv
code, field = _sample_code('future_min')
start, end = _date_range('future_min', code, field, prefer_time=True)
freq = (db['future_min'].find_one({'code': code}, {'type': 1}) or {}).get('type', '1min') if code else '1min'
if not code or not start or not end:
    print('skip: insufficient data in future_min')
else:
    fetch_future_min_adv(code, start, end, frequence=freq)


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_future_list_adv
# new: wequant.wefetch.fetch_future_list_adv
fetch_future_list_adv()


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_block_adv
# new: wequant.wefetch.fetch_stock_block_adv
code, _ = _sample_code('stock_block')
if not code:
    print('skip: no code in stock_block')
else:
    fetch_stock_block_adv(code)


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_realtime_adv
# new: wequant.wefetch.fetch_stock_realtime_adv
print('skip: realtime_* collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_financial_report_adv
# new: wequant.wefetch.fetch_financial_report_adv
print('skip: financial/calendar/divyield collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_financial_calendar_adv
# new: wequant.wefetch.fetch_stock_financial_calendar_adv
print('skip: financial/calendar/divyield collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_stock_divyield_adv
# new: wequant.wefetch.fetch_stock_divyield_adv
print('skip: financial/calendar/divyield collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_cryptocurrency_day_adv
# new: wequant.wefetch.fetch_cryptocurrency_day_adv
print('skip: cryptocurrency collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_cryptocurrency_min_adv
# new: wequant.wefetch.fetch_cryptocurrency_min_adv
print('skip: cryptocurrency collections not present')


In [ ]:
# old: QAFetch/QAQuery_Advance.py.QA_fetch_cryptocurrency_list_adv
# new: wequant.wefetch.fetch_cryptocurrency_list_adv
coll = _ensure_collection('cryptocurrency_list')
if coll is None:
    print('skip: cryptocurrency_list collection not present')
else:
    doc = coll.find_one({}, {'market': 1, 'exchange': 1}) or {}
    market = doc.get('market') or doc.get('exchange')
    if not market:
        print('skip: cryptocurrency_list missing market field')
    else:
        fetch_cryptocurrency_list_adv(market)
